In [1]:
from nanograd import DeviceType, tensor
import numpy as np

создание тензоров

In [2]:
%%timeit
a = tensor.create_data(np.arange(1000), [1000], DeviceType.CPU)

43.7 μs ± 526 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


Проходка по графу вычислений

In [3]:
a = tensor.create_data([2, 2, 2], [3], DeviceType.CPU)
b = tensor.create_data([3, 3, 3], [3], DeviceType.CPU)
c = tensor.create_data([1, 1, 1], [3], DeviceType.CPU)
d = (a * b) + c
upstream_grad = tensor.create_data(np.ones(3), [3], DeviceType.CPU)
d.backward(upstream_grad)
a.grad(), b.grad(), c.grad()

(array([3., 3., 3.], dtype=float32),
 array([2., 2., 2.], dtype=float32),
 array([1., 1., 1.], dtype=float32))

In [4]:
a.data(), b.data(), c.data()

(array([2., 2., 2.], dtype=float32),
 array([3., 3., 3.], dtype=float32),
 array([1., 1., 1.], dtype=float32))

очистка градиента (работает на таргет тензор)

In [5]:
a.zero_grad()
a.grad(), c.grad(), b.grad()

(array([0., 0., 0.], dtype=float32),
 array([1., 1., 1.], dtype=float32),
 array([2., 2., 2.], dtype=float32))

Вычисление на cuda

In [6]:
a = tensor.create_data([2, 2, 2], [3], DeviceType.CUDA)
b = tensor.create_data([3, 3, 3], [3], DeviceType.CUDA)
c = tensor.create_data([1, 1, 1], [3], DeviceType.CUDA)
d = (a * b) + c
upstream_grad = tensor.create_data(np.ones(3), [3], DeviceType.CUDA)
d.backward(upstream_grad)
a = a.to(DeviceType.CPU)
b = b.to(DeviceType.CPU)
c = c.to(DeviceType.CPU)
a.grad(), b.grad(), c.grad()

(array([3., 3., 3.], dtype=float32),
 array([2., 2., 2.], dtype=float32),
 array([1., 1., 1.], dtype=float32))

Вспомогательные функции

In [7]:
a.shape()

[3]

In [8]:
a.device()

<DeviceType.CPU: 0>

In [9]:
a = a.to(DeviceType.CUDA)
a.device()

<DeviceType.CUDA: 1>

In [10]:
a.numel()

3

Доп примеры

In [17]:
a = tensor.create_data([2, 2, 2, 2, 2, 2], [2, 3], DeviceType.CPU)
a.data()

array([2., 2., 2., 2., 2., 2.], dtype=float32)

In [18]:
b = tensor.create_data([3, 3, 3, 3, 3, 3], [2, 3], DeviceType.CPU)
b.data()

array([3., 3., 3., 3., 3., 3.], dtype=float32)

In [23]:
(a + b).data(), (a * b).data(), (a / b).data()

(array([5., 5., 5., 5., 5., 5.], dtype=float32),
 array([6., 6., 6., 6., 6., 6.], dtype=float32),
 array([0.6666667, 0.6666667, 0.6666667, 0.6666667, 0.6666667, 0.6666667],
       dtype=float32))

In [25]:
a.sum(0).data(), a.sum(1).data()

(array([4., 4., 4.], dtype=float32), array([6., 6.], dtype=float32))

In [27]:
c = a.sum(0).sum(0)
c.data()

array([12.], dtype=float32)

без upstream градиентов поскольку скаляр

In [28]:
c.backward()